## Sharing Tutorial: Securely Collaborating in Graphistry

Investigtions are better together. This tutorial walks through the new PyGraphistry method `.privacy()`, which enables API control of the new sharing features

We walk through:

* Global defaults for `graphistry.privacy(mode='private', ...)`
* Compositional per-visualization settings via `g.privacy(...)`
* Inviting and notifying via `privacy(invited_users=[{...])`

### Setup

You need pygraphistry 0.20.0+ for a corresponding Graphistry server (2.37.20+)

In [4]:
#! pip install --user -q graphistry pandas

     |████████████████████████████████| 88 kB 4.9 MB/s 


In [2]:
import graphistry, pandas as pd
graphistry.__version__

'0.20.0'

In [3]:
graphistry.register(
    api=3, username='myuser', password='mypass',
    #protocol='http', server='my.private-server.com'
)

In [4]:
#demo data
g = graphistry.edges(pd.DataFrame({
    's': ['a', 'b', 'c'],
    'd': ['b', 'c', 'a'],
    'v': [1, 1, 2]
}), 's', 'd')

g = g.settings(url_params={'play': 0})

## Safe default: Unlisted & owner-editable

When creating a plot, Graphistry creates a dedicated URL with the following rules:

* Viewing: Unlisted - Only those given the link can access it
* Editing: Owner-only

The URL is unguessable, and the only webpage it is listed at is the creator's private gallery: https://hub.graphistry.com/datasets/ . That means it is as private as whomever the owner shares the URL with.

In [12]:
public_url = g.plot(render=False)

## Switching to fully private by default

Call `graphistry.privacy()` to default to stronger privacy. It sets:

* `mode='private'` - viewing only by owners and invitees
* `invited_users=[]` - no invitees by default
* `notify=False` - no email notifications during invitations
* `message=''`

By default, this means an explicit personal invitation is necessary for viewing. Subsequent plots in the session will default to this setting.

You can also explicitly set or override those as optional parameters.

In [14]:
graphistry.privacy()
# or equivaently, graphistry.privacy(mode='private', invited_users=[], notify=False, message='')

owner_only_url = g.plot(render=False)

## Local overrides

We can locally override settings, such as opting back in to public sharing for some visualizations:

In [15]:
public_g = g.privacy(mode='public')

public_url1 = public_g.plot(render=False)

#Ex: Inheriting public_g's mode='public'
public_g2 = public_g.name('viz2')
public_url2 = public_g.plot(render=False)

#Ex: Global default was still via .privacy()
still_private_url = g.plot(render=False)

## Invitations and notifications

As part of the settings, we can permit specific individuals as viewers or editors, and optionally, send them an email notification







In [18]:
VIEW = '10'
EDIT = '20'

shared_g = g.privacy(
    mode='private',
    notify=True,
    invited_users=[{'email': 'partner1@site1.com', 'action': VIEW},
                   {'email': 'partner2@site2.org', 'action': EDIT}],
    message='Check out this graph!')
shared_url = shared_g.plot(render=False)

The options can be configured globally or locally, just as we did with `mode`. For example, we might not want to send emails by default, just on specific plots:


In [19]:
graphistry.privacy(
    mode='private',
    notify=False,
    invited_users=[{'email': 'partner1@site1.com', 'action': VIEW},
                   {'email': 'partner2@site2.org', 'action': EDIT}])


shared_url = g.plot(render=False)
notified_and_shared_url = g.privacy(notify=True).plot(render=False)

Even if we do not explicitly notify recipients, the objects will still appear in their gallery at https://hub.graphistry.com/datasets/